# Near duplicate detection

Near-duplicate detection in texts refers to the process of identifying documents or text segments that are almost, but not exactly, the same. These near-duplicates might differ in terms of a few words, punctuation, formatting, or minor rephrasing, but they convey very similar or identical information.

Applications of near duplicate detection include:
- Search Engines: To improve diversity of search results by filtering out duplicate pages that have substantially the same content.
- Plagiarism Detection: To identify instances where a text might have been copied with minor modifications.
- Data Deduplication: In large data repositories, it's important to avoid storing multiple near-identical versions of the same document. This is especially important in machine learning as duplicate texts may lead to imbalanced datasets.

# Loading OpenAI API key

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Path to the .env file in the parent directory
env_path = Path("..") / ".env"

# Load environment variables from .env file
load_dotenv(dotenv_path=env_path)

# Get the OpenAI API key from the environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

# The dataset

The dataset we'll be using comes from Kaggle and can be found [here](https://www.kaggle.com/datasets/stackoverflow/statsquestions). This dataset contains questions and answers from the cross-validated stack exchange (which is the machine learning equivilant of stack overflow). I've created a smaller version of this dataset that contains just the question titles. We'll use near-duplicate detection to find duplicate questions based on their titles. Let's dive in!

In [222]:
import pandas as pd

df = pd.read_csv('../datasets/cross-validated/questions.csv')

# High-level approach
To achieve near duplicate detection using text embeddings we'll:
- Retrieve embeddings for each of texts
- Find the n most similar segments for each segment using cosine similarity and the faiss for efficiency
- Record which segment pairs have similarity higher than some threshold
- Remove duplicates

# Retrieving OpenAI embeddings

In [3]:
import openai
import concurrent
import numpy as np
from tqdm import tqdm

def get_embedding(text, model="text-embedding-ada-002"):
    return openai.Embedding.create(input=[text], model=model)['data'][0]['embedding']


def parallel_embedding(text_list, model="text-embedding-ada-002"):
    # Create a ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
        # Submit tasks to the executor and remember the order
        futures = []
        mapping = dict()
        for i, text in tqdm(enumerate(text_list)):
            future = executor.submit(get_embedding, text, model)
            futures.append(future)
            mapping[future] = i

        # Retrieve results as they become available and sort their order
        embeddings = [None] * len(futures)
        for future in tqdm(concurrent.futures.as_completed(futures)):
            embeddings[mapping[future]] = future.result()
    return np.array(embeddings)

# Calculating costs
Embedding text is not free - it costs $0.0001 per 1,000 tokens embedded using text-embedding-ada-002. To ensure that we don't bankrupt ourselves let's calculate the cost of making these api calls to embed the question titles.

> **_NOTE:_**  Api calls are rate limited to 3,500 requests per minute for paid users after 48 hours. That's just more than 58 per second. We'll limit the number of the threads in the threadpool to achieve this however in a production setting you should consider a more robust method of rate limiting to avoid rate limiting errors.

In [238]:
import tiktoken
import numpy


def count_tokens(text: str, model_name: str = "text-embedding-ada-002") -> int:
    enc = tiktoken.encoding_for_model(model_name)
    tokens = enc.encode(text)
    return len(tokens)


total_tokens = np.sum(df['title'].apply(count_tokens))
model_cost = 0.0001 # text-embedding-ada-002 costs $0.0001 for 1,000 tokens

print(f'There are {total_tokens} tokens in the title column of the dataset')
print(f'Total cost of embeddings is ${total_tokens / 1000 * model_cost}')

There are 941510 tokens in the title column of the dataset
Total cost of embeddings is $0.094151


Thankfully the cost is less than \\$0.10 but if we had tried to embed the full question descriptions we would be looking at a significantly higher bill (probably around \\$10).

In [5]:
embeddings = parallel_embedding(df['title'])

85085it [00:01, 79177.49it/s]
85085it [34:01, 41.67it/s]


# Efficient similarity search using faiss by Facebook
Faiss contains several methods for similarity search including euclidean distance and dot product. We'd like to use cosine similarity, which happens to be the same as the dot product on normalised vectors. We'll then create our search index, normalise the embeddings and add them to the index.

In [101]:
import faiss


def normalize(embeddings):
    return embeddings / np.linalg.norm(embeddings, axis=1)[:, np.newaxis]


dimension = embeddings_normalized.shape[1]
index = faiss.IndexFlatL2(dimension)
embeddings_normalized = normalize(embeddings)
index.add(embeddings_normalized)

In [132]:
def search(i, embeddings_normalized, n=10):
    query_embedding = np.array([embeddings_normalized[i]])
    # Search for the closest embeddings
    distances, ind = index.search(query_embedding, n)
    # Convert distances to cosine similarity
    cosine_sim = 1 - distances / 2
    # Return 1-d arrays
    return ind[0], cosine_sim[0]

In [135]:
from sklearn.metrics.pairwise import cosine_similarity
import time

i = 2
ind, sim = search(i, embeddings_normalized)
pd.DataFrame(data={'title': df['title'][ind], 'sim': sim})

,title,sim
2,Bayesian and frequentist reasoning in plain En...,1.000000
24045,Bayesian vs frequentist Interpretations of Pro...,0.923964
7617,Interpretation of Bayesian vs Frequentist stat...,0.917631
75922,bayesian analysis of frequentist methods,0.911131
77335,Bayesian vs. frequentist estimation,0.910255
82312,Frequentist statistics,0.904825
55708,Am I understanding differences between Bayesia...,0.904304
36876,An example of the differences in frequentist a...,0.904229
24936,Bayesian and frequentist interpretations vs ap...,0.902857
77975,Bayesian vs. Frequentist calculation steps,0.902417


# Finding near duplicates
Now we have an efficient way to search our embeddings using cosine similairty lets find the top 10 most similar titles, for each of our titles. We can then filter out any that do not meet a given similarity threshold. We'll then remove duplicate matches as well as any matches from one title to itself.

In [153]:
from tqdm import tqdm

def find_near_duplicates(embeddings_normalized, threshold, n=10):
    arrays = []
    similarities = []

    for i in tqdm(range(len(embeddings_normalized))):
        ind, sim = search(i, embeddings_normalized, n=n)
        a = np.zeros((n, 2)).astype(int)
        a[:, 0] = i
        a[:, 1] = ind
        arrays.append(a)
        similarities.append(sim)

    # Concatenate arrays and similarities
    concatenated_arrays = np.vstack(arrays)
    concatenated_similarities = np.concatenate(similarities)

    # Filter out rows that do not meet the threshold
    mask = concatenated_similarities >= threshold
    concatenated_arrays = concatenated_arrays[mask]
    concatenated_similarities = concatenated_similarities[mask]

    # Remove duplicate matches and matches between the same items
    mask = concatenated_arrays[:, 0] < concatenated_arrays[:, 1]
    concatenated_arrays = concatenated_arrays[mask]
    concatenated_similarities = concatenated_similarities[mask]

    return concatenated_arrays, concatenated_similarities

In [154]:
a, sim = find_near_duplicates(embeddings_normalized, 0.95)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85085/85085 [22:05<00:00, 64.17it/s]


In [221]:
duplicates = pd.DataFrame(
    data = {
        'title_left': df.title.values[a[:, 0]],
        'ind_left': a[:, 0],
        'title_right': df.title.values[a[:, 1]],
        'ind_right': a[:, 1],
        'similarity': sim,
    }
)
duplicates

,title_left,ind_left,title_right,ind_right,similarity
0,Posterior distribution,40323,Posterior distribution,69304,1.000000
1,Tests of within subjects contrasts in R,11685,Tests of within subjects contrasts in R,12957,1.000000
2,Priors and Loss in R,46962,Priors and Loss in R,69759,1.000000
3,Convert SAS syntax for mixed models for repeat...,68468,Convert SAS syntax for mixed models for repeat...,80367,1.000000
4,Convergence in distribution,24294,Convergence in distribution,28155,1.000000
...,...,...,...,...,...
3747,Multiple Regression Model vs Multivariate Model,51868,Multiple Versus Multivariate Regression,54461,0.950009
3748,Calculating probability,59367,How to compute probability,75141,0.950008
3749,"Standard error of proportions, with weighting",33441,Standard error of proportion,46926,0.950007
3750,Interpreting ACF and PACF Plot,25207,Help to interpret unusual ACF/PACF plots,43558,0.950004


# Removing the duplicates

Reviewing the 3,752 paired entries, we've identified a wealth of potential duplicate candidates. There is no doubt that cross-validated do a significant amount of work to ensure that duplicate questions are detected and removed so this dataset is already very clean.

To go a step further we could add the long descriptions of questions into the embedding step to incorporate more semantic information, however, for the sake of this tutorial (and my wallet) I have kept things simple.

As an additional step we could send this subset of results of for human evaluation or we could leverage a large language model to verify that duplicates are truly asking the same questions. For simplicity we'll assume that all the near-duplicates are true duplicates. Let's go ahead and remove them.

In [236]:
# Create a boolean index
mask = np.ones(len(df)).astype(bool)
# Mark the duplicates for removal
mask[a[:, 1]] = False
# Apply the mask
df = df.loc[mask, :]

In [232]:
df

,id,title,score
0,0,The Two Cultures: statistics vs. machine learn...,272
1,1,Forecasting demographic census,4
2,2,Bayesian and frequentist reasoning in plain En...,208
3,3,What is the meaning of p values and t values i...,138
4,4,Examples for teaching: Correlation does not me...,58
...,...,...,...
85079,85079,Modeling linear regression with covariate depe...,0
85080,85080,Interpretation of global test (p-value) of the...,0
85082,85082,How do I know a simple validation result is st...,1
85083,85083,Kendall conditional independence test statistic,1
